In [15]:
import torch
import evaluate
import os
import csv
from tqdm import tqdm

def load_texts(file_path, source_file_path):
    """
    Load texts from a file and return as a list of stripped strings.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        #also read  source file to get the source text
        with open(source_file_path, 'r', encoding='utf-8') as f_source:
            line = [line.strip() for line in f.readlines()]
            line_source = [line.strip() for line in f_source.readlines()]
            #the source line contains a # , save the content before and after the # in two different vars
            line_source = [line.split('#') for line in line_source]
            #remove both elements of the list line_source from the line of the hypothesis
            for i in range(len(line_source)):
                line[i] = line[i].replace(line_source[i][0], '')
                line[i] = line[i].replace(line_source[i][1], '')
                print(line_source[i][1])
                print(line[i])

        return line

def main():
    # Define explicit pairs of hypothesis and target files
    # Format: (hypothesis_file_path, target_file_path)
    file_pairs = [
        ("generated_hypotheses/ikatdata_checkpoint16.hypo", "D3test/ikatdata.target", "D3test/ikatdata.source"),
        ("generated_hypotheses/ikatdata_checkpoint50.hypo", "D3test/ikatdata.target", "D3test/ikatdata.source"),
        ("generated_hypotheses/jandata_checkpoint16.hypo", "D2test/jandata.target", "D2test/jandata.source"),
        ("generated_hypotheses/jandata_checkpoint50.hypo", "D2test/jandata.target", "D2test/jandata.source"),
        ("generated_hypotheses/semevaldata_checkpoint16.hypo", "D1test/semevaldata.target", "D1test/semevaldata.source"),
        ("generated_hypotheses/semevaldata_checkpoint50.hypo", "D1test/semevaldata.target", "D1test/semevaldata.source")
    ]
    
    # Output CSV file to store evaluation results
    output_csv = 'evaluation_results.csv'
    
    # Initialize metrics
    bleu = evaluate.load('bleu')
    bertscore = evaluate.load('bertscore')
    rouge = evaluate.load('rouge')  # Optional, if ROUGE is desired
    
    # Prepare CSV header
    csv_header = ['Model', 'Source File', 'BLEU-1', 'BLEU-2', 
                  'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 
                  'BERTScore Precision', 'BERTScore Recall', 'BERTScore F1']
    
    # Open CSV file for writing
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)
        
        print(f"Starting evaluation of {len(file_pairs)} file pairs.\n")
        
        # Iterate over each file pair with a progress bar
        for hypo_path, target_path, source_path in tqdm(file_pairs, desc="Evaluating file pairs"):
            # Extract model identifier and source file name from the hypothesis file name
            hypo_filename = os.path.basename(hypo_path)
            try:
                sourcefile, modelid = hypo_filename.split('_checkpoint')
                modelid = f"checkpoint{modelid.split('.hypo')[0]}"
            except Exception as e:
                print(f"Filename format error for {hypo_filename}: {e}")
                continue  # Skip this pair if filename is not as expected
            
            # Check if both files exist
            if not os.path.exists(hypo_path):
                print(f"Hypothesis file not found: {hypo_path}. Skipping.")
                continue
            if not os.path.exists(target_path):
                print(f"Target file not found: {target_path}. Skipping.")
                continue
            
            # Load texts
            generated_hypo_texts = load_texts(hypo_path, source_path)
            target_texts = load_texts(target_path, source_path)
            
            # Validate that all datasets have the same number of samples
            if len(target_texts) != len(generated_hypo_texts):
                print(f"Line count mismatch between hypothesis and target for {hypo_filename}. Skipping.")
                continue  # Skip if line counts do not match
            
            # Compute BLEU-1
            bleu1 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=1
            )
            
            # Compute BLEU-2
            bleu2 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=2
            )
            
            # Compute ROUGE
            ro = rouge.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                use_stemmer=True
            )
            
            # Compute BERTScore
            bs = bertscore.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                lang='en',
                model_type='bert-base-uncased',
                verbose=False
            )
            
            # Calculate average BERTScore metrics
            bert_precision = sum(bs['precision']) / len(bs['precision']) * 100
            bert_recall = sum(bs['recall']) / len(bs['recall']) * 100
            bert_f1 = sum(bs['f1']) / len(bs['f1']) * 100
            
            # Write results to CSV
            writer.writerow([
                modelid,
                sourcefile,
                f"{bleu1['bleu'] * 100:.2f}",
                f"{bleu2['bleu'] * 100:.2f}",
                f"{ro['rouge1'] * 100:.2f}",
                f"{ro['rouge2'] * 100:.2f}",
                f"{ro['rougeL'] * 100:.2f}",
                f"{bert_precision:.2f}",
                f"{bert_recall:.2f}",
                f"{bert_f1:.2f}"
            ])
            
            # Print summary for this pair
            print(f"Evaluated {hypo_filename}: BLEU-1={bleu1['bleu'] * 100:.2f}, BLEU-2={bleu2['bleu'] * 100:.2f}, ROUGE-1={ro['rouge1'] * 100:.2f}, ROUGE-2={ro['rouge2'] * 100:.2f}, ROUGE-L={ro['rougeL'] * 100:.2f}, BERTScore F1={bert_f1:.2f}")
        
        print(f"\nEvaluation completed. Results saved to {output_csv}.")

# Execute the evaluation
main()


Starting evaluation of 6 file pairs.



Evaluating file pairs:   0%|          | 0/6 [00:00<?, ?it/s]

 computer games should also not be recognized as Olympic events.
And since She didn't know how to be a lot of money. also should also be not be recognized as Olympic events.
 computer games should also not be recognized as Olympic events.
And since He didn't know how to try it. also should also be not be recognized as Olympic events.
 Supermarkets and shopping centres should be allowed to open on any Sundays and holidays.
Considering the growing digitization of society, the traditional model of the weekend. And since He didn't want to go to the store to get a lot of time to be a lot of time to be a good weekend to be in the house. shoppingmarkets and shopping should be allowed to be allowed to be allowed to be allowed to open on open on any Sundays and open any Sundays.
 they're very bad however, if the rent suddenly climbs by 100,- or 200,-.
If there must be rent increases, there should also be a cap. And since There was a new cap. they're very bad however, if the rent suddenly by 100

Evaluating file pairs:   0%|          | 0/6 [00:00<?, ?it/s]


KeyboardInterrupt: 